# Constrained Optimization of the Full Transmon Model

In [11]:
import os
from os.path import join
import sys
import numpy as np
import matplotlib.pylab as plt

$\newcommand{\Op}[1]{\boldsymbol{\mathsf{\hat{#1}}}}$

## Introduction

We consider the full Transmon Hamiltonian
\begin{equation}
  \Op{H} = \sum_{q=1,2} \left[ \omega_q \Op{b}^{\dagger}_q \Op{b}_q
    + \frac{\alpha_q}{2} \Op{b}^{\dagger}_q\Op{b}^{\dagger}_q\Op{b}_q\Op{b}_q
    + g_q \left(\Op{b}^{\dagger}_q \Op{a} + \Op{b}_q \Op{a}^{\dagger} \right)
    \right]
    + \omega_c \Op{a}^{\dagger}\Op{a}
    + \epsilon^{*}(t) \Op{a} + \epsilon(t) \Op{a}^{\dagger}
\end{equation}
Note that unlike in the optimizations done for KITP in 2013, we do *not* have a static qubit-qubit coupling, outside the effective coupling that is mediated by the cavity

Goals:

1.  Scan the parameter regime systematically for the point that allows the
    easiest implementation of both single- and two-qubit gates. Identify
    known gate mechanisms on map. The scan shows:
    *   Which parameters yield the fastest gates
    *   Which local equivalence classes different qubit parameters lead to
    *   Whether there are "good" regions in addition to the regions at which known gate mechanism have been implemented (determined by resonance conditions)

2.  For select points, determine the quantum speed limit for high fidelity
    gates: What are the fastest possible quantum gates with all-microwave control of transmon qubits 
3.  Include robustness with respect to dissipation, and fluctuations in the
    qubit parameters

Plan:

*   Prerequisite: Method testing
    *   Redo a direct gate optimization for KITP parameters, but for $J=0$.
    *   Test optimization with state-dependent and frequency constraints
    *   Test optimization with perfect entangler
    *   Test optimization for single-qubit gate
    *   Examine the RIP method for depopulating the cavity. If generally applicable, incorporate into all guess pulses below
    
*   Scan parameter space.

    We choose $\omega_1$, $\alpha_1 \neq \alpha_2$, $g_1 = g_2$ constant. We vary $\omega_2$ and $\omega_c$ (in the RWA) on a grid. At each grid point (set of paremters), we perform the following steps to evaluate the parameters as a candidate:
    
    *   Set $T=200$ ns (possibly also $T = 500$ ns and $T = 100$ ns
    *   Determine a good guess pulse for the optimization towards a perfect entangler, and for the optimization towards a single-qubit gate
    *   Perform a Krotov or LBFGS optimization, for PE and for single qubit gates (either LI-optimization for identity, or two optimizations $\sigma_x^{(1)}$ and $\sigma_x^{(2)}$.
    
    The critical step is the determination of a good guess pulse. A possiblity is to generate simple guess pulses based on some randomized parameters, and to perform a few simplex steps in these parameters.
    
*  Pick several points in the parameter space that yielded the best success. Determin the QSL by optimizing at increasingly smaller values of $T$.
    

## Environment Setup

We keep our Python installation, as well as all compiled code in a virtual environment, stored in `$PREFIX`. The source code for the QDYN library and the trasmon propgram has been checked out to `$PREFIX/src/qdyn` and `$PREFIX/src/transmon_oct`, respectively.

    #!/bin/bash
    conda create -p ./venv anaconda

    export PREFIX=`pwd`/venv
    export LD_LIBRARY_PATH=$PREFIX/lib:$LD_LIBRARY_PATH
    export PATH=$PREFIX/bin:$PATH

    mkdir -p ./venv/src

    (cd ./venv/src/ && git clone git@jerusalem.physik.uni-kassel.de:qdyn)
    (cd ./venv/src/ && git clone git@jerusalem.physik.uni-kassel.de:goerz/transmon_oct)

    (cd $PREFIX/src/qdyn && git checkout master && ./configure --prefix=$PREFIX --no-hooks && make install)
    (cd $PREFIX/src/transmon_oct && git checkout master && ./configure --prefix=$PREFIX && make install)

    pip install -I git+ssh://git@jerusalem.physik.uni-kassel.de/qdynpylib@master#egg=QDYN
    pip install -I git+ssh://git@jerusalem.physik.uni-kassel.de/goerz/QDYNTransmonLib@master#egg=QDYNTransmonLib
    pip install mgplottools

In [3]:
%install_ext http://raw.github.com/goerz/version_information/master/version_information.py
%load_ext version_information
% version_information numpy, matplotlib, pandas, QDYN, QDYNTransmonLib, "cd $PREFIX/src/qdyn && git log",  "cd $PREFIX/src/transmon_oct && git log"

Installed version_information.py. To use it, type:
  %load_ext version_information


Software,Version
Python,2.7.8 64bit [GCC 4.4.7 20120313 (Red Hat 4.4.7-1)]
IPython,2.2.0
OS,Linux 2.6.32 504.8.1.el6.x86_64 x86_64 with redhat 6.2 Zoot
numpy,1.9.0
matplotlib,1.4.0
pandas,0.14.1
QDYN,2.0.dev1
QDYNTransmonLib,1.0
cd $PREFIX/src/qdyn && git log,commit 5360e2415fb20a98dbe87df2a15691825e177f47
cd $PREFIX/src/transmon_oct && git log,commit 47bba1c8e394a17fda452105a66d71be80c4b713


In [9]:
PREFIX = join(os.getcwd(), 'venv')
assert os.path.samefile(join(PREFIX, 'bin'), os.environ['PATH'].split(os.pathsep)[0])

## Re-examination of KITP parameters and Application of new Methods

In [12]:
params = {
    'w_c': 8.3, # GHZ
    'w_1': 6.5, # GHZ
    'w_2': 6.6, # GHz
    'alpha_1': -300, # GHZ
    'alpha_2': -300, # MHZ
    'g_1'    :  100, # MHZ
    'g_2'    :  100 # MHZ
}

### Are there any resonances hidden in these parameters?

[TODO: plot the first 3 or 4 levels lowest-lying levels]

### How much entanglement is generated in the field-free case?

[TODO]

### How much entanglement is generated by Blackman pulses of varying intensity?

[TODO]

### Is the RWA justified?

[TODO: propagate guess pulse with and without RWA, compare results]

## Prerequisite: Does Optimization with State-Dependent Constraints Work?

We want to penalize cavity excitation. We can enforce this in either one of two ways:
*   State-Dependent constraints, with inhomogeneous backward propagation
*   A non-Hermitian Hamiltonian

For both cases, we should ensure that a single backward and forward propagation runs without crashing and gives reasonable results.

For a single set of parameters and gate durations, we run an unconstrained optimization and a constrained optimization, and compare the resulting pulses. We could use either direct optimization, or PE optimizaton